<a href="https://colab.research.google.com/github/vladimirsnap/options/blob/main/Volatility_calc_IEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/OPT-SPY.csv')
df.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3397 entries, 0 to 3396
Data columns (total 27 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       3397 non-null   object 
 1   stockpx    3397 non-null   float64
 2   iv30       3397 non-null   float64
 3   iv60       3397 non-null   float64
 4   iv90       3397 non-null   float64
 5   m1atmiv    3397 non-null   float64
 6   m1dtex     3397 non-null   float64
 7   m2atmiv    3397 non-null   float64
 8   m2dtex     3397 non-null   float64
 9   m3atmiv    3397 non-null   float64
 10  m3dtex     3397 non-null   float64
 11  m4atmiv    3397 non-null   float64
 12  m4dtex     3397 non-null   float64
 13  slope      3397 non-null   float64
 14  deriv      3397 non-null   float64
 15  slope_inf  3397 non-null   float64
 16  deriv_inf  3397 non-null   float64
 17 

In [ ]:
# Выберите столбцы с IV различных периодов
df_iv = df[['iv30', 'iv60', 'iv90']]

# Вычисление корреляции
correlation_matrix = df_iv.corr()

print(correlation_matrix)

          iv30      iv60      iv90
iv30  1.000000  0.988600  0.976427
iv60  0.988600  1.000000  0.992888
iv90  0.976427  0.992888  1.000000


In [ ]:
df.describe()

,stockpx,iv30,iv60,iv90,m1atmiv,m1dtex,m2atmiv,m2dtex,m3atmiv,m3dtex,...,10dclsHV,20dclsHV,60dclsHV,120dclsHV,252dclsHV,10dORHV,20dORHV,60dORHV,120dORHV,252dORHV
count,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,...,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000
mean,246.194095,15.645202,16.277468,16.702953,17.041180,16.533412,16.144975,47.130998,16.549208,77.826023,...,13.804012,14.528413,15.398304,15.919500,16.539806,14.739500,15.011790,15.556647,15.993456,16.530683
std,101.848031,6.538722,5.826157,5.403632,19.747461,9.707523,6.175012,10.449169,5.620937,10.744876,...,9.591804,9.169519,8.209176,7.301958,6.170599,9.001565,8.541115,7.545303,6.624742,5.533474
min,102.200000,6.390000,7.800000,8.350000,1.070000,2.000000,6.690000,30.000000,7.920000,58.000000,...,1.900000,3.110000,4.940000,6.240000,6.720000,3.520000,4.320000,5.420000,6.460000,6.960000
25%,161.370000,11.310000,12.340000,12.890000,11.230000,9.000000,12.080000,39.000000,12.720000,69.000000,...,7.760000,8.850000,10.750000,11.570000,12.050000,9.060000,9.630000,10.890000,11.580000,12.350000
50%,216.750000,14.040000,14.830000,15.400000,14.350000,17.000000,14.610000,47.000000,15.200000,76.000000,...,11.460000,12.370000,13.210000,13.730000,15.230000,12.370000,12.740000,13.480000,14.060000,15.600000
75%,307.100000,18.300000,18.940000,19.350000,19.480000,24.000000,18.770000,54.000000,19.130000,86.000000,...,17.190000,17.800000,17.950000,18.180000,18.770000,17.810000,18.120000,18.720000,18.760000,19.610000
max,477.710000,75.310000,70.440000,65.070000,1000.000000,62.000000,75.390000,97.000000,70.640000,125.000000,...,106.520000,91.260000,61.530000,45.930000,33.910000,94.310000,79.140000,54.640000,41.180000,31.160000


In [ ]:
import numpy as np
from scipy.signal import argrelextrema

# Указываем период для скользящей средней
period = 30

# Рассчитываем скользящую среднюю IV
df['IV_MA'] = df['iv30'].rolling(window=period).mean()

# Находим локальные максимумы и минимумы
df['local_max'] = df.iloc[argrelextrema(df['iv30'].values, np.greater)[0]]['iv30']
df['local_min'] = df.iloc[argrelextrema(df['iv30'].values, np.less)[0]]['iv30']

# Инициализируем колонки для дней до возврата к средней
df['days_to_mean_from_max'] = np.nan
df['days_to_mean_from_min'] = np.nan

# Рассчитываем количество дней до возврата к средней для максимумов
max_days = df[df['local_max'].notna()].index
for day in max_days:
    try:
        df.loc[day, 'days_to_mean_from_max'] = df[(df.index > day) & (df['iv30'] <= df.loc[day, 'IV_MA'])].index[0] - day
    except IndexError:
        pass

# Рассчитываем количество дней до возврата к средней для минимумов
min_days = df[df['local_min'].notna()].index
for day in min_days:
    try:
        df.loc[day, 'days_to_mean_from_min'] = df[(df.index > day) & (df['iv30'] >= df.loc[day, 'IV_MA'])].index[0] - day
    except IndexError:
        pass


In [ ]:
# Посмотреть общую статистику по дням до возврата к средней для максимумов
print(df['days_to_mean_from_max'].describe())

# Посмотреть общую статистику по дням до возврата к средней для минимумов
print(df['days_to_mean_from_min'].describe())

# Посмотреть среднее количество дней до возврата к средней для максимумов
print(df['days_to_mean_from_max'].mean())

# Посмотреть среднее количество дней до возврата к средней для минимумов
print(df['days_to_mean_from_min'].mean())



count    879.000000
mean      16.501706
std       46.771676
min        1.000000
25%        1.000000
50%        1.000000
75%        7.000000
max      344.000000
Name: days_to_mean_from_max, dtype: float64
count     875.000000
mean       20.116571
std       128.859289
min         1.000000
25%         1.000000
50%         1.000000
75%         8.000000
max      2145.000000
Name: days_to_mean_from_min, dtype: float64
16.501706484641637
20.11657142857143


In [ ]:
df['change_from_max_to_mean'] = df['local_max'] - df['IV_MA']
df['change_from_min_to_mean'] = df['IV_MA'] - df['local_min']


In [ ]:
print(df['change_from_max_to_mean'].describe())
print(df['change_from_min_to_mean'].describe())


count    883.000000
mean       1.092163
std        4.074655
min      -28.741667
25%       -0.674833
50%        0.805000
75%        2.637167
max       32.452000
Name: change_from_max_to_mean, dtype: float64
count    886.000000
mean       0.924097
std        3.601440
min      -23.108000
25%       -0.539583
50%        0.677333
75%        2.156667
max       31.845667
Name: change_from_min_to_mean, dtype: float64


In [ ]:
df.describe()

,stockpx,iv30,iv60,iv90,m1atmiv,m1dtex,m2atmiv,m2dtex,m3atmiv,m3dtex,...,60dORHV,120dORHV,252dORHV,IV_MA,local_max,local_min,days_to_mean_from_max,days_to_mean_from_min,change_from_max_to_mean,change_from_min_to_mean
count,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,3397.000000,...,3397.000000,3397.000000,3397.000000,3368.000000,890.000000,893.000000,879.000000,875.000000,883.000000,886.000000
mean,246.194095,15.645202,16.277468,16.702953,17.041180,16.533412,16.144975,47.130998,16.549208,77.826023,...,15.556647,15.993456,16.530683,15.656495,16.808843,14.741221,16.501706,20.116571,1.092163,0.924097
std,101.848031,6.538722,5.826157,5.403632,19.747461,9.707523,6.175012,10.449169,5.620937,10.744876,...,7.545303,6.624742,5.533474,5.867647,7.410390,6.160630,46.771676,128.859289,4.074655,3.601440
min,102.200000,6.390000,7.800000,8.350000,1.070000,2.000000,6.690000,30.000000,7.920000,58.000000,...,5.420000,6.460000,6.960000,6.961000,6.720000,6.390000,1.000000,1.000000,-28.741667,-23.108000
25%,161.370000,11.310000,12.340000,12.890000,11.230000,9.000000,12.080000,39.000000,12.720000,69.000000,...,10.890000,11.580000,12.350000,11.868750,11.902500,10.710000,1.000000,1.000000,-0.674833,-0.539583
50%,216.750000,14.040000,14.830000,15.400000,14.350000,17.000000,14.610000,47.000000,15.200000,76.000000,...,13.480000,14.060000,15.600000,14.190667,14.955000,13.260000,1.000000,1.000000,0.805000,0.677333
75%,307.100000,18.300000,18.940000,19.350000,19.480000,24.000000,18.770000,54.000000,19.130000,86.000000,...,18.720000,18.760000,19.610000,18.391917,19.567500,17.140000,7.000000,8.000000,2.637167,2.156667
max,477.710000,75.310000,70.440000,65.070000,1000.000000,62.000000,75.390000,97.000000,70.640000,125.000000,...,54.640000,41.180000,31.160000,48.998667,75.310000,67.390000,344.000000,2145.000000,32.452000,31.845667


In [ ]:
# Рассчитываем скользящее среднее и скользящее стандартное отклонение IV30
df['rolling_mean_iv30'] = df['iv30'].expanding().mean()
df['rolling_std_iv30'] = df['iv30'].expanding().std()

# Создаем новый DataFrame для хранения результатов
df_iv30_2std = pd.DataFrame(columns=['date', 'return_to_mean_date', 'price_change'])

# Перебираем все даты
for date in df['date']:
    # Если на данную дату IV30 отклоняется от среднего на 2 и более стандартных отклонений...
    if abs(df.loc[df['date'] == date, 'iv30'].values[0] - df.loc[df['date'] == date, 'rolling_mean_iv30'].values[0]) >= 2 * df.loc[df['date'] == date, 'rolling_std_iv30'].values[0]:
        # ... то находим первую дату после этого, когда IV30 возвращается к среднему
        return_to_mean_date = df[(df['date'] > date) & (df['iv30'] <= df['rolling_mean_iv30'])]['date'].min()

        # Если такая дата существует, рассчитываем изменение цены акции
        if return_to_mean_date is not None and not df[df['date'] == return_to_mean_date].empty:
            price_at_2std_deviation = df[df['date'] == date]['stockpx'].values[0]
            price_at_return_to_mean = df[df['date'] == return_to_mean_date]['stockpx'].values[0]
            price_change = price_at_return_to_mean - price_at_2std_deviation

            # Добавляем информацию в df_iv30_2std
            new_row = pd.DataFrame({'date': [date], 'return_to_mean_date': [return_to_mean_date],
                                    'price_change': [price_change]})
            df_iv30_2std = pd.concat([df_iv30_2std, new_row], ignore_index=True)

# Выводим первые строки df_iv30_2std
print(df_iv30_2std.head())


         date return_to_mean_date  price_change
0  2023-06-15          2023-06-22         -6.09
1  2023-06-01          2023-06-08          7.31
2  2023-05-31          2023-06-08         11.28
3  2023-05-30          2023-06-08          8.95
4  2023-05-26          2023-06-08          9.11


In [ ]:
# Рассчитываем среднее и стандартное отклонение IV30
df['mean_iv30'] = df['iv30'].expanding().mean()
df['std_iv30'] = df['iv30'].expanding().std()

# Создаем новый DataFrame для хранения результатов
df_iv30_2std = pd.DataFrame(columns=['date', 'return_to_mean_date', 'price_change', 'price_change_percent'])

# Перебираем все даты
for date in df['date']:
    # Если на данную дату IV30 отклоняется от среднего на 2 и более стандартных отклонений...
    if abs(df.loc[df['date'] == date, 'iv30'].values[0] - df.loc[df['date'] == date, 'mean_iv30'].values[0]) >= 2 * df.loc[df['date'] == date, 'std_iv30'].values[0]:
        # ... то находим первую дату после этого, когда IV30 возвращается к среднему
        return_to_mean_date = df[(df['date'] > date) & (df['iv30'] <= df['mean_iv30'])]['date'].min()

        # Если такая дата существует, рассчитываем изменение цены акции
        if return_to_mean_date is not None and not df[df['date'] == return_to_mean_date].empty:
            price_at_2std_deviation = df[df['date'] == date]['stockpx'].values[0]
            price_at_return_to_mean = df[df['date'] == return_to_mean_date]['stockpx'].values[0]
            price_change = price_at_return_to_mean - price_at_2std_deviation
            price_change_percent = (price_change / price_at_2std_deviation) * 100

            # Добавляем информацию в df_iv30_2std
            new_row = pd.DataFrame({'date': [date], 'return_to_mean_date': [return_to_mean_date],
                                    'price_change': [price_change], 'price_change_percent': [price_change_percent]})
            df_iv30_2std = pd.concat([df_iv30_2std, new_row], ignore_index=True)

# Выводим первые строки df_iv30_2std
print(df_iv30_2std.head())


         date return_to_mean_date  price_change  price_change_percent
0  2023-06-15          2023-06-22         -6.09             -1.375960
1  2023-06-01          2023-06-08          7.31              1.732967
2  2023-05-31          2023-06-08         11.28              2.699533
3  2023-05-30          2023-06-08          8.95              2.130040
4  2023-05-26          2023-06-08          9.11              2.168944
